In [1]:
# Lotus is your AI friend that plays simple text based games with you

# !pip install python-dotenv
import os
import openai
from dotenv import load_dotenv

# **BURAYI .env DOSYANIZIN YERI OLACAK SEKILDE DEGISTIRIN***
dotenv_path = 'key.env'
is_loaded = load_dotenv(dotenv_path)

# Load config values

# Setting up the deployment name
chatgpt_model_name = os.getenv('CHATGPT_MODEL')

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv('OPENAI_API_BASE')

# Currently Chat Completions API have the following versions available: 2023-03-15-preview
openai.api_version = os.getenv('OPENAI_API_VERSION')

deployment_name = os.getenv('DEPLOYMENT_NAME')


print("Are environment variables loaded correctly? ", is_loaded)

Are environment variables loaded correctly?  True


# Yerleşik LangChain araçlarını kullanma: DuckDuckGo arama ve Wikipedia

In [2]:
#!pip install -U wikipedia



from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name=chatgpt_model_name,
    model_name=chatgpt_model_name,
    temperature=0.5,
    max_tokens=500,
    top_p=0.9,
    frequency_penalty=0,
    presence_penalty=0,
)

tools = load_tools(["llm-math","wikipedia"], llm=llm)

agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

agent("300'ün %25'i kaçtır?")

question = "Tom M. Mitchell, Amerikalı bir bilgisayar bilimcisi ve \
Carnegie Mellon Üniversitesi'nde (CMU) Founders Üniversitesi Profesörü, \
hangi kitabı yazdı?"

result = agent(question)



> Entering new  chain...
Thought: I should use the calculator tool for this question.

Action:
```
{
    "action": "Calculator",
    "action_input": "300*25/100"
}
```


Observation: Answer: 75.0
Thought: I now know the final answer.

Final Answer: 75.0<|im_end|>

> Finished chain.


> Entering new  chain...
Thought: I need to search who is Tom M. Mitchell and what book he wrote
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```


c:\Users\cbark\Desktop\AI\projects\myenv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\cbark\Desktop\AI\projects\myenv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.



Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules foo

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


 Tom M. Mitchell wrote the book Machine Learning
Final Answer: Machine Learning<|im_end|>

> Finished chain.


# Python Ajanı

In [3]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

agent.run(f"""Bu müşterileri soyadına ve ardından adına göre sıralayın ve \
 çıktıyı yazdırın: {customer_list}""")



> Entering new  chain...
 I can make a list of the customers and use the sort function to sort them
Action: Python_REPL
Action Input: customers = [['Jen', 'Ayai'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too']]
customers.sort(key=lambda x: (x[1], x[0]))
print(customers)
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought: The list is sorted in the correct order
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]<|im_end|>

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]<|im_end|>"

# Zincirlerin ayrıntılı çıktılarını görüntüleyelim

In [4]:
import langchain

langchain.debug=True

agent.run(f"""Bu müşterileri soyadına ve ardından adına göre sıralayın ve \
 çıktıyı yazdırın: {customer_list}""")

langchain.debug=False

[chain/start] [1:RunTypeEnum.chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Bu müşterileri soyadına ve ardından adına göre sıralayın ve  çıktıyı yazdırın: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:RunTypeEnum.chain:AgentExecutor > 2:RunTypeEnum.chain:LLMChain] Entering Chain run with input:
{
  "input": "Bu müşterileri soyadına ve ardından adına göre sıralayın ve  çıktıyı yazdırın: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:RunTypeEnum.chain:AgentExecutor > 2:RunTypeEnum.chain:LLMChain > 3:RunTypeEnum.llm:AzureOpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to write and execute python code to answer questions.\nYou ha

# Kendi aracımızı tanımlayalım

In [5]:
#!pip install DateTime

from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Bugünün tarihini döndürür, bugünün tarihini bilmekle ilgili tüm sorular \
       için bunu kullanın. \
      Girdi her zaman boş bir dize olmalıdır ve bu işlev her zaman bugünün \
     tarihini döndürür - herhangi bir tarih matematiği bu işlevin dışında \
     gerçekleşmelidir."""
    return str(date.today())

agent= initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Not:**
Temsilci bazen yanlış sonuca varabilir (ajanlar'ın geliştirilmesine devam edilmektedir!). Varsa, lütfen tekrar çalıştırmayı deneyin.

In [6]:
try:
    result = agent("Bugünün tarihi ne?")
except:
    print("Harici erişimde istisna gerçekleşti")



> Entering new  chain...
Thought: I should use the `time` tool to get today's date
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-06-20
Thought: I now know today's date
Final Answer: 2023-06-20<|im_end|>

> Finished chain.
